In [1]:
from pyspark import SparkContext, SparkConf, SparkContext

try: 
    sc.stop()
except:
    print("sc have not yet created!")
conf = SparkConf().setAppName("First App").setMaster("spark://master:7077")
sc = SparkContext(conf=conf)
print(sc)

sc have not yet created!
<SparkContext master=spark://master:7077 appName=First App>


# Create Seasion

In [2]:
# khoi tao seasion
from pyspark.sql import SparkSession, SQLContext
# my_spark = SparkSession.builder.getOrCreate()
my_spark = SparkSession(sc)
print(my_spark)

In [3]:
# show all data in my_spark 
my_spark.catalog.listTables()

[]

# Mở file lưu trên HDFS

In [4]:
data = my_spark.read.csv('hdfs://localhost:9000/user/input/flights.csv', header=True)
airport = my_spark.read.csv('airports.csv', header=True)
airlines = my_spark.read.csv('airlines.csv', header=True)

In [5]:
data.show(5)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [6]:
# show info schema
data.printSchema()

root
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: string (nullable = true)
 |-- ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: string (nullable = true)
 |-- D

In [7]:
# Them bang bảng vao catalog

data.createOrReplaceTempView('Flights')
airlines.createOrReplaceTempView('Airlines')
airport.createOrReplaceTempView('Airports')
my_spark.catalog.listTables()

[Table(name='airlines', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='airports', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

# Thao tac SQL tren Dataframe và spark.sql

In [8]:
# Đếm số lượng các chuyến bay trong tuần

import time
t1 = time.time()
data.groupBy('DAY_OF_WEEK')\
    .count().alias("NUM_OF_WEEK")\
    .sort("DAY_OF_WEEK")\
    .withColumnRenamed("count", "NUM_OF_WEEK")\
    .show()
print(time.time()-t1)

+-----------+-----------+
|DAY_OF_WEEK|NUM_OF_WEEK|
+-----------+-----------+
|          1|     865543|
|          2|     844600|
|          3|     855897|
|          4|     872521|
|          5|     862209|
|          6|     700545|
|          7|     817764|
+-----------+-----------+

5.991016864776611


In [9]:
t1 = time.time()
my_spark.sql("""SELECT DAY_OF_WEEK, count('DAY_OF_WEEK') AS NUM_OF_WEEK 
                FROM Flights 
                GROUP BY DAY_OF_WEEK 
                ORDER BY DAY_OF_WEEK
                """).show()
print(time.time()-t1)

+-----------+-----------+
|DAY_OF_WEEK|NUM_OF_WEEK|
+-----------+-----------+
|          1|     865543|
|          2|     844600|
|          3|     855897|
|          4|     872521|
|          5|     862209|
|          6|     700545|
|          7|     817764|
+-----------+-----------+

4.10817551612854


In [10]:
# Hiển thị các chuyến bay với khoảng cách lớn hơn 1000
my_spark.sql("""SELECT AIRLINE, FLIGHT_NUMBER, ORIGIN_AIRPORT, DESTINATION_AIRPORT, DISTANCE
                FROM Flights
                WHERE DISTANCE>1000
             """)\
        .show(10)



+-------+-------------+--------------+-------------------+--------+
|AIRLINE|FLIGHT_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DISTANCE|
+-------+-------------+--------------+-------------------+--------+
|     AS|           98|           ANC|                SEA|    1448|
|     AA|         2336|           LAX|                PBI|    2330|
|     US|          840|           SFO|                CLT|    2296|
|     AA|          258|           LAX|                MIA|    2342|
|     AS|          135|           SEA|                ANC|    1448|
|     DL|          806|           SFO|                MSP|    1589|
|     NK|          612|           LAS|                MSP|    1299|
|     US|         2013|           LAX|                CLT|    2125|
|     AA|         1112|           SFO|                DFW|    1464|
|     DL|         1173|           LAS|                ATL|    1747|
+-------+-------------+--------------+-------------------+--------+
only showing top 10 rows



In [24]:
# Hiển thị các chuyến bay từ của hàng AS cất cánh từ LAX
df = my_spark.sql("""
                SELECT * FROM Flights
                WHERE (AIRLINE == 'AS') AND (ORIGIN_AIRPORT == 'LAX')
            """)
df.show(5)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-